# Comparing Trained and Evaluated Model Experiments Using Vertex AI Experiments

## Overview

As a Data Scientist, you probably start running model experiments locally on your notebook. Depending on the framework you use, you would need to track parameters, training time series and evaluation metrics. In this way, you would be able to explain the modelling approach you would choose. 

### Lab objectives

In this notebook, you learn how to use Vertex AI Experiments to compare and evaluate model experiments.

This notebook uses the following Google Cloud ML services and resources:

- Vertex AI Workbench
- Vertex AI Experiments

The steps performed include:

- Log the model parameters.
- Log the loss and metrics on every epoch to TensorBoard.
- Log the evaluation metrics.
- Compare two experiments.

in Vertex AI Experiment of a recurrent neural network (RNN) for sentiment analysis.

Each learning objective will correspond to a #TODO in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/comparing_local_trained_models.ipynb).

### Install additional packages

Install additional package dependencies not installed in your notebook environment, such as TensorFlow, TensorFlow Datasets and Vertex AI SDK. Use the latest major GA version of each package.

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade tensorflow==2.8.0 tensorflow_datasets==4.5.2 -q
! pip3 install --user --force-reinstall 'google-cloud-aiplatform>=1.15' -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.9.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.55.0 which is incompatible.
tfx-bsl 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 9.0.0 which is incompatible.
tfx-bsl 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<3,>=1.15.5, but you have tensorflow 2.8

**Note**: Please ignore the incompatibility error and warnings.

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  qwiklabs-gcp-01-babac012e883


Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [3]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. It is recommended that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [4]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this notebook.

In [5]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [7]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [8]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [9]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://qwiklabs-gcp-01-babac012e883aip-20220818154737/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [10]:
! gsutil ls -al $BUCKET_URI

### Import libraries

In [11]:
# General
import logging

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

# Vertex AI
from google.cloud import aiplatform as vertex_ai

### Define constants

In [12]:
# Training
DATA_DIR = "data"
LOG_DIR = "logs"

# Experiments
TASK = "classification"
MODEL_TYPE = "rnn"
EXPERIMENT_NAME = f"{PROJECT_ID}-{TASK}-{MODEL_TYPE}-{TIMESTAMP}"

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [13]:
vertex_ai.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### Create TensorBoard instance using Vertex AI TensorBoard

You can upload your TensorBoard logs by first creating a TensorBoard instance.

Learn more about [TensorBoard overview](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview).

In [14]:
vertex_ai_tb = vertex_ai.Tensorboard.create()

Creating Tensorboard


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Creating Tensorboard


Create Tensorboard backing LRO: projects/745211089335/locations/us-central1/tensorboards/2410111895898947584/operations/5087686935921557504


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Create Tensorboard backing LRO: projects/745211089335/locations/us-central1/tensorboards/2410111895898947584/operations/5087686935921557504


Tensorboard created. Resource name: projects/745211089335/locations/us-central1/tensorboards/2410111895898947584


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Tensorboard created. Resource name: projects/745211089335/locations/us-central1/tensorboards/2410111895898947584


To use this Tensorboard in another session:


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:To use this Tensorboard in another session:


tb = aiplatform.Tensorboard('projects/745211089335/locations/us-central1/tensorboards/2410111895898947584')


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:tb = aiplatform.Tensorboard('projects/745211089335/locations/us-central1/tensorboards/2410111895898947584')


In [15]:
vertex_ai.init(experiment=EXPERIMENT_NAME, experiment_tensorboard=vertex_ai_tb)

## Training with Vertex AI Experiments

Vertex AI enables users to track the steps (for example, preprocessing, training) of an experiment run, and track inputs (for example, algorithm, parameters, datasets) and outputs (for example, models, checkpoints, metrics) of those steps. 

Below you have some example of how track experiments to train recurrent neural network for sentiment analysis. 

To simplify the code, here you have helper function to cover the following steps:

- Collect training data
- Create text encoder
- Build a RNN as baseline model
- Build a LSTM as challenger model
- Train the model

In [16]:
!mkdir -p -m 777 $DATA_DIR $LOG_DIR

In [17]:
import tensorflow_datasets as tfds

tfds.disable_progress_bar()
import tensorflow as tf


# Helpers ----------------------------------------------------------------------
def get_dataset(data_dir, buffer_size, batch_size):
    """
    Returns a tf.data.Dataset object containing the training data
    Returns:
        tf.data.Dataset: A tf.data.Dataset object containing the training data
        buffer_size: The buffer size for prefetch data
        batch_size: The batch size of dataset
    """

    dataset = tfds.load(
        "imdb_reviews",
        download=True,
        data_dir=data_dir,
        with_info=False,
        as_supervised=True,
    )

    train_dataset, test_dataset = dataset["train"], dataset["test"]

    train_dataset = (
        train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    )
    test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return train_dataset, test_dataset


def get_encoder(train_dataset, vocab_size):
    """
    Returns a TextVectorization object for the encoder
    Args:
        train_dataset: A tf.data.Dataset object containing the training data
    Returns:
        TextVectorization: A TextVectorization object for the encoder
    """

    encoder = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
    encoder.adapt(
        train_dataset.map(lambda text, label: text, num_parallel_calls=tf.data.AUTOTUNE)
    )
    return encoder


def get_baseline_model(encoder, model_params):
    """
    Returns a tf.keras.Model object for the model as baseline
    Args:
        encoder: A TextVectorization object for the encoder
        model_params: A dictionary with model parameters
    Returns:
        tf.keras.Model: A tf.keras.Model object for the model
    """

    model = tf.keras.Sequential()
    model.add(encoder)
    model.add(
        tf.keras.layers.Embedding(
            input_dim=len(encoder.get_vocabulary()), output_dim=64, mask_zero=True
        )
    )
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
    model.add(tf.keras.layers.Dense(64, activation="relu"))
    model.add(tf.keras.layers.Dense(1))
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=model_params["learning_rate"],
            beta_1=model_params["beta_1"],
            beta_2=model_params["beta_2"],
            epsilon=model_params["epsilon"],
        ),
        metrics=["accuracy"],
    )
    return model


def get_stacked_model(encoder, model_params):
    """
    Returns a tf.keras.Model object for the model as challenger
        Args:
            encoder: A TextVectorization object for the encoder
            model_params: A dictionary with model parameters
        Returns:
            tf.keras.Model: A tf.keras.Model object for the model
    """

    model = tf.keras.Sequential()
    model.add(encoder)
    model.add(
        tf.keras.layers.Embedding(
            input_dim=len(encoder.get_vocabulary()), output_dim=64, mask_zero=True
        )
    )
    model.add(
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))
    )
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
    model.add(tf.keras.layers.Dense(64, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1))
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=model_params["learning_rate"],
            beta_1=model_params["beta_1"],
            beta_2=model_params["beta_2"],
            epsilon=model_params["epsilon"],
        ),
        metrics=["accuracy"],
    )
    return model


def train(train_dataset, test_dataset, model, epochs, steps, log_dir=LOG_DIR):
    """
    Trains the model
    Args:
        train_dataset: A tf.data.Dataset object containing the training data
        test_dataset: A tf.data.Dataset object containing the testing data
        model: A tf.keras.Model object for the model
        epochs: The number of epochs
        steps: The number of validation steps
        log_dir: The location of tf training logs
    """
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    history = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=epochs,
        validation_steps=steps,
        callbacks=[tensorboard],
    )
    return history

### Define a baseline

As part of the experimentation phase, you would like to define a baseline model to have a performance benchmark which it is represented by recurrent neural network (RNN) in this case. Then you try to improve the ML solution with more complex training configuration that produce better results.

#### Run experiment and evaluate experiment runs using `with` statement

This step would takes **15 min** approx. to finish. And it covers the following steps:

- Initialize an experiment run
- Log the parameters associated to training data
- Log the parameters of the encoder
- Log the parameters of the model
- Train the model
- Log the metrics for each epochs
- Log the overall training metrics

In [18]:
# Experiment Settings ----------------------------------------------------------
RUN_ID_1 = "run-1"
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VOCAB_SIZE = 1000
EPOCHS = 2
STEPS = 20
ROLE = "baseline"
LR = 1e-4
B_1 = 0.9
B_2 = 0.999
EPS = 1e-07

# Initialize the experiment
logging.info("Initialize the experiment.")
with vertex_ai.start_run(RUN_ID_1) as run:

    # Get the training and testing datasets
    logging.info("Get the training and testing datasets.")
    data_params = {"buffer_size": BUFFER_SIZE, "batch_size": BATCH_SIZE}
    train_dataset, test_dataset = get_dataset(
        "./data",
        buffer_size=data_params["buffer_size"],
        batch_size=data_params["batch_size"],
    )
    run.log_params(data_params)

    # Get the encoder
    logging.info("Get the encoder.")
    encoder_params = {"vocab_size": VOCAB_SIZE}
    encoder = get_encoder(
        train_dataset=train_dataset, vocab_size=encoder_params["vocab_size"]
    )
    run.log_params(encoder_params)

    # TODO 1: Get the model
    logging.info("Get the model.")
    run.log_params({"role": ROLE})
    model_params = # Your code goes here
    model = get_baseline_model(encoder=encoder, model_params=model_params)
    run.log_params(model_params)

    # TODO 2: Train the model
    logging.info("Train the model.")
    history = train(
        # Your code goes here
    )

    run.log_params(history.params)
    for idx in range(0, history.params["epochs"]):
        run.log_time_series_metrics(
            {
                "train_loss": history.history["loss"][idx],
                "train_accuracy": history.history["accuracy"][idx],
            }
        )

    # Evaluate model
    logging.info("Evaluate model.")
    test_loss, test_accuracy = model.evaluate(test_dataset)
    run.log_metrics({"test_loss": test_loss, "test_accuracy": test_accuracy})

    # Exit the experiment
    logging.info("Exit the run.")

INFO:root:Initialize the experiment.


Associating projects/745211089335/locations/us-central1/metadataStores/default/contexts/qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737-run-1 to Experiment: qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/745211089335/locations/us-central1/metadataStores/default/contexts/qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737-run-1 to Experiment: qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737
INFO:root:Get the training and testing datasets.
INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: imdb_reviews/plain_text/1.0.0
INFO:absl:Load dataset info from /tmp/tmpsu0ftwictfds
INFO:absl:Field info.release_notes from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Field in

INFO:absl:Done writing data/imdb_reviews/plain_text/1.0.0.incompleteRF0TE4/imdb_reviews-train.tfrecord*. Number of examples: 25000 (shards: [25000])
INFO:absl:Done writing data/imdb_reviews/plain_text/1.0.0.incompleteRF0TE4/imdb_reviews-test.tfrecord*. Number of examples: 25000 (shards: [25000])
INFO:absl:Done writing data/imdb_reviews/plain_text/1.0.0.incompleteRF0TE4/imdb_reviews-unsupervised.tfrecord*. Number of examples: 50000 (shards: [50000])
INFO:absl:Constructing tf.data.Dataset imdb_reviews for split None, from ./data/imdb_reviews/plain_text/1.0.0
2022-08-18 15:52:02.495965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-08-18 15:52:02.496016: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (3

Dataset imdb_reviews downloaded and prepared to ./data/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


...

2022-08-18 15:58:48.255852: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


391/391 [==============================] - 390s 997ms/step - loss: 0.3790 - accuracy: 0.8296 - val_loss: 0.3695 - val_accuracy: 0.8164


2022-08-18 16:05:18.400660: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
INFO:root:Evaluate model.


391/391 [==============================] - 92s 234ms/step - loss: 0.3467 - accuracy: 0.8319


INFO:root:Exit the run.


#### Evaluate the baseline model

In [19]:
# Get experiment
logging.info("Get experiment status.")
experiment_df = vertex_ai.get_experiment_df()
experiment_df.T

INFO:root:Get experiment status.


,0
experiment_name,qwiklabs-gcp-01-babac012e883-classification-rn...
run_name,run-1
run_type,system.ExperimentRun
state,COMPLETE
param.role,baseline
param.learning_rate,0.0001
param.steps,391.0
param.vocab_size,1000.0
param.verbose,1.0
param.beta_2,0.999


In [20]:
# Get time series metrics
logging.info("Get time series metrics.")
ts_runs_df = run.get_time_series_data_frame()
ts_runs_df

INFO:root:Get time series metrics.


,step,wall_time,train_accuracy,train_loss
0,1,2022-08-18 16:05:19.552000+00:00,0.58260,0.630458
1,2,2022-08-18 16:05:19.674000+00:00,0.82956,0.378996


### Make experimental improvement to the model

As challenger model you add a LSTM layer to the baseline model. This step would takes **20 min** approx. to finish.


#### Run experiment and evaluate experiment runs with `experiment` instance


In [21]:
# Experiment Settings ----------------------------------------------------------
RUN_ID_2 = "run-2"
ROLE = "stacked"

# Initialize the experiment
logger.info("Initialize the experiment.")
vertex_ai.start_run(RUN_ID_2)

# Get the model
logging.info("Get the model.")
run.log_params({"role": ROLE})
model = get_stacked_model(encoder=encoder, model_params=model_params)
vertex_ai.log_params(model_params)

# Train the model
logging.info("Train the model.")
history = train(
    model=model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    epochs=EPOCHS,
    steps=STEPS,
)
vertex_ai.log_params(history.params)
for idx in range(0, history.params["epochs"]):
    vertex_ai.log_time_series_metrics(
        {
            "train_loss": history.history["loss"][idx],
            "train_accuracy": history.history["accuracy"][idx],
        }
    )

# TODO 3: Evaluate model
logging.info("Evaluate model.")
test_loss, test_accuracy = # Your code goes here
vertex_ai.log_metrics({"test_loss": test_loss, "test_accuracy": test_accuracy})

# Exit the experiment
logging.info("Exit the experiment.")
vertex_ai.end_run()

INFO:logger:Initialize the experiment.


Associating projects/745211089335/locations/us-central1/metadataStores/default/contexts/qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737-run-2 to Experiment: qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/745211089335/locations/us-central1/metadataStores/default/contexts/qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737-run-2 to Experiment: qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737
INFO:root:Get the model.
INFO:root:Train the model.


Epoch 1/2
391/391 [==============================] - 681s 2s/step - loss: 0.6096 - accuracy: 0.6070 - val_loss: 0.5632 - val_accuracy: 0.7547
Epoch 2/2
391/391 [==============================] - 667s 2s/step - loss: 0.3809 - accuracy: 0.8335 - val_loss: 0.3894 - val_accuracy: 0.8227


INFO:root:Evaluate model.


391/391 [==============================] - 157s 402ms/step - loss: 0.3583 - accuracy: 0.8334


INFO:root:Exit the experiment.


#### Compare the baseline with the new model implementation

In [22]:
# TODO 4: Get experiment
logging.info("Get experiment status.")
experiment_df = # Your code goes here
experiment_df.T

INFO:root:Get experiment status.


,0,1
experiment_name,qwiklabs-gcp-01-babac012e883-classification-rn...,qwiklabs-gcp-01-babac012e883-classification-rn...
run_name,run-2,run-1
run_type,system.ExperimentRun,system.ExperimentRun
state,COMPLETE,COMPLETE
param.beta_2,0.999,0.999
param.verbose,1.0,1.0
param.steps,391.0,391.0
param.epsilon,0.0,0.0
param.learning_rate,0.0001,0.0001
param.beta_1,0.9,0.9


In [23]:
# Get time series metrics
exp_run = vertex_ai.ExperimentRun(RUN_ID_2, experiment=EXPERIMENT_NAME)
logging.info("Get time series metrics.")
ts_runs_df = exp_run.get_time_series_data_frame()
ts_runs_df

INFO:root:Get time series metrics.


,step,wall_time,train_loss,train_accuracy
0,1,2022-08-18 17:11:29.356000+00:00,0.609591,0.60700
1,2,2022-08-18 17:11:29.634000+00:00,0.380914,0.83348


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the notebook.

Otherwise, you can delete the individual resources you created in this notebook.

In [24]:
# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
exp.delete(delete_backing_tensorboard_runs=True)

# Delete Tensorboard
vertex_ai_tb.delete()

# Delete Cloud Storage objects that were created
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}

!rm -Rf $DATA_DIR $LOG_DIR

Deleting TensorboardRun : projects/745211089335/locations/us-central1/tensorboards/2410111895898947584/experiments/qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737/runs/run-2


INFO:google.cloud.aiplatform.base:Deleting TensorboardRun : projects/745211089335/locations/us-central1/tensorboards/2410111895898947584/experiments/qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737/runs/run-2


Delete TensorboardRun  backing LRO: projects/745211089335/locations/us-central1/tensorboards/2410111895898947584/experiments/qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737/operations/2539775446736699392


INFO:google.cloud.aiplatform.base:Delete TensorboardRun  backing LRO: projects/745211089335/locations/us-central1/tensorboards/2410111895898947584/experiments/qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737/operations/2539775446736699392


TensorboardRun deleted. . Resource name: projects/745211089335/locations/us-central1/tensorboards/2410111895898947584/experiments/qwiklabs-gcp-01-babac012e883-classification-rnn-20220818154737/runs/run-2


...

Delete Tensorboard  backing LRO: projects/745211089335/locations/us-central1/operations/4516855683152347136


INFO:google.cloud.aiplatform.base:Delete Tensorboard  backing LRO: projects/745211089335/locations/us-central1/operations/4516855683152347136


Tensorboard deleted. . Resource name: projects/745211089335/locations/us-central1/tensorboards/2410111895898947584


INFO:google.cloud.aiplatform.base:Tensorboard deleted. . Resource name: projects/745211089335/locations/us-central1/tensorboards/2410111895898947584


Removing gs://qwiklabs-gcp-01-babac012e883aip-20220818154737/...


In [25]:
!rm -Rf $DATA_DIR $LOG_DIR